<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/inventory_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar dependencias en Colab
!pip install torch==2.0.1 transformers==4.31.0 sentencepiece

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Verificación de GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Usando dispositivo:", device)

# Cargar un modelo Transformer (GPT-2)
model_name = "gpt2"  # Puedes usar variantes más grandes como gpt2-medium, etc.
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
model.eval()

# ------------------------------------------------------------
# Ejemplo de datos sintéticos de inventario:
# Supongamos que tienes datos diarios de inventario (unidades en stock),
# por ejemplo, representadas como una serie de números.
# Por simplicidad, convertiremos los números a texto y el modelo intentará
# continuar la secuencia.
#
# En la vida real, deberías:
# 1) Convertir tus datos de inventario en una secuencia textual estructurada.
# 2) Probar distintos formateos (prompt engineering) que ayuden al modelo
#    a entender la secuencia temporal.
# 3) Ajustar un modelo con fine-tuning si es necesario.
# ------------------------------------------------------------

# Generamos datos sintéticos de inventario (por ejemplo, 30 días de datos):
import random
random.seed(42)
historical_inventories = [1000]  # Inventario inicial
for i in range(1, 30):
    # Simulamos variaciones diarias aleatorias
    next_val = max(0, historical_inventories[-1] + random.randint(-50, 50))
    historical_inventories.append(next_val)

# Creamos un prompt que el modelo recibirá:
# Ej: "Inventario diario (en unidades): Día 1: 1000, Día 2: 980, Día 3: 1020, ..."
# Luego le pedimos que prediga el siguiente día.
prompt_text = "Inventario diario (en unidades): " + ", ".join([f"Día {i+1}: {inv}" for i,inv in enumerate(historical_inventories)]) + ", Día 31:"

# Tokenizamos el prompt
input_ids = tokenizer.encode(prompt_text, return_tensors='pt').to(device)

# Generamos el texto completado por el modelo
# Aquí el modelo va a "continuar" la secuencia.
# Ajusta max_length según la cantidad de pasos que quieras predecir.
max_length = input_ids.shape[1] + 10  # Intentemos predecir unos tokens más adelante
with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=max_length,
        temperature=0.7,      # Ajustable, controla la variabilidad
        top_p=0.9,            # Ajustable, controla el filtrado por rango acumulado
        do_sample=True,       # Si hacemos sampling para mayor diversidad
        eos_token_id=tokenizer.eos_token_id
    )

# Decodificamos el resultado
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Prompt original:\n", prompt_text)
print("\nTexto generado:\n", generated_text)

# Nota: El modelo GPT-2 no sabe que son datos de inventario,
# simplemente continúa la secuencia textual. Podrás ver que
# "inventa" valores o texto. Para un forecasting real,
# necesitas:
# - Entrenar con datos similares
# - Ajustar el prompt
# - Usar un modelo especializado en series temporales
# - O Implementar un Transformer entrenado con librerías
#   especializadas (ej. Darts, PyTorch Forecasting, etc.)
